In [1]:
import sys
import numpy as np
import pandas as pd
sys.path.append('..')
from main import BlockAnalysis
import matplotlib.pyplot as plt

In [2]:
time, rg, bias = np.loadtxt('cv_bias.dat',unpack=True)

### Error estimation for time-correlated data series

In [ ]:
'''
The variable rg contains the time trace of the radius
of gyration of a peptide obtained from an MD simulation. 

With the multi keyword you indicate that the rg array
is made from the concatenation of 2 indipendent trajectoy.
The result is that there will not be blocks covering the 
end of a trajectory and the beginning of the other. 
'''
block_rg = BlockAnalysis(rg, multi=2)

In [ ]:
'''
After initializing the class we can call the statistics
from the block averaging
'''
print(pd.DataFrame(block_rg.stat, columns=['Block size', 'SEM', 'err(SEM)']))
plt.errorbar(block_rg.stat[...,0], block_rg.stat[...,1], block_rg.stat[...,2], fmt='', color='k', ecolor='0.5')
plt.xlabel('Block size')
plt.ylabel('SEM')

In [ ]:
'''
To avoid manually picking the point in the block profile
at the beginning of the plateau (decorrelating block length,
where the SEM have the less uncertainty), we can rely on
automatic recognition of the decorrelating block length.
'''
block_rg.SEM()
print('Mean:', block_rg.av)
print('SEM:', block_rg.sem)
print('Decorrelating block length:', block_rg.bs)
plt.errorbar(block_rg.stat[...,0], block_rg.stat[...,1], block_rg.stat[...,2], fmt='', color='k', ecolor='0.5')
plt.scatter(block_rg.bs, block_rg.sem,zorder=10,c='tab:red')
plt.xlabel('Block size')
plt.ylabel('SEM')

### Error estimation for the free-energy of biased time-correlated data series (MetaD)

In [ ]:
'''
The dataset loaded in the 2nd cell comes from a WTMetaD run where the
rg is a biased CV. In this case we perform the block analysis on
the FES, rather than on the CV.

The class can take either the Boltzmann weights in input, or MetaD bias
and temperature and estimate the weights internally.
'''
block_fes = BlockAnalysis(rg,bias=bias,T=310,multi=2,interval_low=0.6,interval_up=2.5)

In [ ]:
print(pd.DataFrame(block_fes.stat, columns=['Block size', 'SEM', 'err(SEM)']))
plt.errorbar(block_fes.stat[...,0], block_fes.stat[...,1], block_fes.stat[...,2], fmt='', color='k', ecolor='0.5')
block_fes.SEM()
plt.scatter(block_fes.bs, block_fes.sem,zorder=10,c='tab:red')
plt.xlabel('Frame #')
plt.ylabel('<SEM>(FES) [kJ/mol]')

In [ ]:
'''
Beware that in this last case the SEM obtained is just an
average of the SEM over the FES. Calculating the average SEM
is only useful to get a decorrelating block size. We can indeed
use that information to obtain histograms and free-energy surfaces
with error bars specific for each point.
'''
binC, H, E = block_fes.get_pdf()

In [ ]:
plt.plot(binC, H, 'k')
plt.fill_between(binC, H-E, H+E)
plt.ylabel('p(Rg)')
plt.xlabel('Rg')

In [ ]:
binC, FES, FES_err = block_fes.get_fes()
plt.plot(binC, FES, 'k')
plt.fill_between(binC, FES-FES_err, FES+FES_err)
plt.ylabel('FES(Rg) (kJ/mol)')
plt.xlabel('Rg')